# Guía 3 — Introducción a LangChain con OpenAI API
Curso: **IA en el Aula — Nivel Avanzado**  
Profesor: **Luis Daniel Benavides Navarro**  
Fecha: **Octubre 2025**

En esta guía aprenderás los conceptos básicos de **LangChain**, una librería diseñada para construir aplicaciones impulsadas por modelos de lenguaje, integrando la API de OpenAI con flujos más complejos. Exploraremos cómo crear un primer **prompt chain**, administrar memoria y usar herramientas para componer respuestas inteligentes.

## 1️⃣ Instalación y configuración inicial
LangChain se instala como cualquier otra librería de Python. También usaremos `python-dotenv` para gestionar la clave de OpenAI. Ejecute la siguiente celda:

In [1]:
%pip install openai langchain python-dotenv langchain-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2️⃣ Cargar variables de entorno y cliente OpenAI
Crea un archivo `.env` con tu clave de API y cárgala para poder usarla dentro de LangChain. Esto evita exponer tu clave en el código fuente.

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()  # Cargar archivo .env
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("No se encontró la clave OPENAI_API_KEY en el archivo .env")

# Crear cliente LangChain con OpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
print("Cliente LangChain con OpenAI inicializado correctamente.")


Cliente LangChain con OpenAI inicializado correctamente.


## 3️⃣ Primer ejemplo: Prompt simple con LangChain
LangChain utiliza **chains** (cadenas de pasos) para procesar información. Comencemos con una cadena simple que envía un mensaje al modelo y obtiene la respuesta.

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the LLM (uses your OpenAI API key from environment)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# Create a simple prompt
prompt = ChatPromptTemplate.from_template(
    "Explica en dos frases el concepto de {tema}."
)

# Combine the components using LCEL (LangChain Expression Language)
chain = prompt | llm | StrOutputParser()

# Run it
result = chain.invoke({"tema": "aprendizaje automático"})
print(result)


El aprendizaje automático es una rama de la inteligencia artificial que permite a las computadoras aprender de datos y mejorar su rendimiento en tareas específicas sin ser programadas explícitamente para cada una de ellas. Utiliza algoritmos y modelos estadísticos para identificar patrones y hacer predicciones basadas en la información proporcionada.


### Explicación
- **ChatPromptTemplate:** define la estructura del mensaje que se envía al modelo.
- **ChatOpenAI:** la conexión real al modelo.
- **chain** crea la cadena de componentes usando LCEL (LangChain Expression Language).
- **StrOutputParser** convierte la salida estructurada del modelo en texto plano.
- **chain.invoke:** ejecuta la cadena pasando los valores del prompt.

Este patrón permite reutilizar prompts para distintos temas o contextos.

## 4️⃣ Ejemplo 2: Encadenar múltiples pasos
LangChain permite combinar varios pasos en una misma ejecución. En este ejemplo, crearemos dos prompts: uno para definir un tema y otro para generar una aplicación educativa basada en el resultado.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# 1) LLM (usa tu OPENAI_API_KEY en el entorno)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
to_str = StrOutputParser()

# 2) Paso 1: explicar brevemente el concepto de {tema}
primer_prompt = ChatPromptTemplate.from_template(
    "Explica brevemente el concepto de {tema}."
)
primer_paso = primer_prompt | llm | to_str
# `primer_paso` produce un string, por ejemplo: "La realidad aumentada es ..."

# 3) Paso 2: proponer una aplicación educativa usando la salida del paso 1 como {concepto}
segundo_prompt = ChatPromptTemplate.from_template(
    "Propón una aplicación educativa del siguiente concepto: {concepto}."
)
segundo_paso = segundo_prompt | llm | to_str

# 4) Encadenar: mapear la entrada {tema} al primer paso, y su salida a {concepto} del segundo
cadena_secuencial = {"concepto": primer_paso} | segundo_paso

# 5) Ejecutar la cadena completa
resultado = cadena_secuencial.invoke({"tema": "realidad aumentada"})
print(resultado)

**Aplicación Educativa: "Exploradores del Conocimiento"**

**Descripción General:**
"Exploradores del Conocimiento" es una aplicación educativa basada en realidad aumentada (RA) diseñada para enriquecer el aprendizaje en diversas materias, como historia, ciencias, matemáticas y arte. La aplicación permite a los estudiantes interactuar con contenido digital superpuesto en su entorno real, facilitando una experiencia de aprendizaje más inmersiva y participativa.

**Características Principales:**

1. **Exploración de Temas:**
   - Los estudiantes pueden seleccionar diferentes temas de estudio (por ejemplo, el sistema solar, civilizaciones antiguas o estructuras matemáticas).
   - Al apuntar su dispositivo hacia un área específica (como una mesa o un mural en el aula), la aplicación proyectará modelos 3D, videos explicativos y gráficos interactivos relacionados con el tema elegido.

2. **Interacción y Aprendizaje Activo:**
   - Los estudiantes pueden interactuar con los elementos virtuales

### Explicación
- **cadena_secuencial:** permite conectar varias cadenas; la salida de una se convierte en la entrada de la siguiente.
- En este caso, el modelo primero explica el tema y luego sugiere una aplicación educativa.

Este tipo de flujo es ideal para generar contenido didáctico o ideas para proyectos en clase.

## 5️⃣ Ejemplo 3: Añadir memoria a la conversación
LangChain incluye módulos de **memoria** para mantener contexto entre múltiples interacciones. Esto permite simular conversaciones educativas más naturales, donde el modelo recuerda temas previos.

In [6]:

# Instalar si hace falta:
# %pip install -U langchain langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser

# LLM (requiere OPENAI_API_KEY en el entorno)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
to_str = StrOutputParser()

# Prompt con hueco para el historial
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente educativo claro y conciso."),
    MessagesPlaceholder("chat_history"),      # ← aquí va la memoria
    ("human", "{input}")
])

# Cadena base
chain = prompt | llm | to_str

# Memoria simple como lista de mensajes
history: list = []

def chat(user_text: str) -> str:
    """
    Envía un turno del usuario, usa el historial y actualiza la memoria
    con el par (usuario, asistente).
    """
    global history
    # Ejecutar la cadena inyectando el historial actual
    answer = chain.invoke({"input": user_text, "chat_history": history})
    # Actualizar memoria (guardar los dos mensajes)
    history += [HumanMessage(content=user_text), AIMessage(content=answer)]
    return answer

# --- Ejemplo de uso (tres turnos) ---
print(chat("Hola, soy un profesor de informática."))
print(chat("¿Puedes explicarme cómo introducir IA a mis estudiantes?"))
print(chat("¿Qué ejemplos prácticos puedo usar en la clase?"))



¡Hola! ¿En qué puedo ayudarte hoy en tu labor como profesor de informática?
Claro, aquí tienes algunas ideas para introducir la inteligencia artificial (IA) a tus estudiantes:

1. **Conceptos Básicos**: Comienza explicando qué es la IA, sus tipos (IA débil vs. IA fuerte) y sus aplicaciones en la vida cotidiana (asistentes virtuales, recomendaciones de productos, etc.).

2. **Historia de la IA**: Proporciona un breve resumen de la evolución de la IA, desde sus inicios hasta los avances recientes. Esto ayudará a los estudiantes a entender el contexto.

3. **Demostraciones Prácticas**: Utiliza herramientas y plataformas accesibles, como Google Teachable Machine o Scratch con extensiones de IA, para mostrar cómo se pueden crear modelos de IA sin necesidad de programación avanzada.

4. **Proyectos Prácticos**: Asigna proyectos donde los estudiantes puedan crear sus propios modelos simples de IA, como un clasificador de imágenes o un chatbot básico. Esto puede hacerse utilizando lenguajes de

### Explicación
- **ConversationBufferMemory:** almacena los mensajes anteriores en la conversación.
- **ConversationChain:** combina el modelo con la memoria.
- Esta funcionalidad es útil para tutores inteligentes o chatbots educativos que requieren continuidad en el diálogo.

## 6️⃣ Buenas prácticas con LangChain + OpenAI
- Usa prompts **claros y estructurados**: el modelo responde mejor cuando la tarea está bien definida.
- Controla `temperature` según la tarea: bajo (0.1–0.3) para precisión, alto (0.7–0.9) para creatividad.
- Guarda logs o historiales si tu aplicación incluye interacción prolongada.
- Limita la longitud de las respuestas con `max_tokens` para evitar costos o respuestas excesivas.
- Documenta tus cadenas (`LLMChain`) para reusarlas en distintos contextos educativos.

## ✅ Conclusión
Has aprendido los fundamentos de LangChain: prompts, chains, memoria y flujo secuencial. Estos conceptos son la base para desarrollar asistentes educativos, tutores personalizados o sistemas de generación de contenido en el aula. En la próxima guía implementaremos un **asistente educativo con RAG (Retrieval-Augmented Generation)** usando tus propios materiales docentes.